# Tile-wise pixel statistics

In [24]:
#%matplotlib inline
import numpy as np
from pathlib import Path
import os
import rasterio as rio
import rasterio.plot as rioplot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

from skimage import data, img_as_float
from skimage import exposure
from PIL import Image
import pandas as pd


harz = [
    '20180419_074323_0c43_3B',
    '20180419_074324_0c43_3B',
    '20180419_074324_1_0c43_3B',
    '20180419_074325_0c43_3B',    
    '20180419_074326_0c43_3B',    
    '20180504_094435_0e19_3B',    
    '20180724_094554_0e19_3B'
]

In [19]:
DATADIR = "../../data/train_raw"
RAW_MAPS = DATADIR + "/map"     #'../data/raw/images'

In [20]:
def getPixels(file):
    with rio.open(file) as inds:
         
        ima = inds.read()

        vals, counts = np.unique(ima, return_counts=True)

        dic = (dict(zip(vals, counts)))
        if 127 not in dic.keys():
            dic[127] = 0
        if 255 not in dic.keys():
            dic[255] = 0
    return  dic

In [5]:
stats = []
for file in Path(RAW_MAPS).iterdir():
    if file.name.endswith(('.tif', '.tiff')):
        dic = getPixels(file)
        stats.append([file.name, dic[0], dic[127], dic[255]])

In [6]:
df = pd.DataFrame.from_dict(stats)
df.columns = ['name', 'numPixel_noRoad', 'numPixel_pavedRoad', 'numPixel_unpavedRoad']

In [7]:
col_list= list(df)
col_list.remove('name')
col_list

['numPixel_noRoad', 'numPixel_pavedRoad', 'numPixel_unpavedRoad']

In [8]:
df['sumPixels'] = df[col_list].sum(axis=1)

In [9]:
df['relative_noRoad']    = df['numPixel_noRoad']/df['sumPixels']
df['relative_pavedRoad'] = df['numPixel_pavedRoad']/df['sumPixels']
df['relative_unpavedRoad'] = df['numPixel_unpavedRoad']/df['sumPixels']

In [32]:
df['region'] = 'Borneo'

for harz_i in harz:
    df[(df.name.str.contains(harz_i))].region = 'Harz'

/home/lisa/.local/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [33]:
np.sum(df.region=='Harz')

0

In [26]:
df.to_csv('numOfPixPerClassPerTile_256.csv')